In [1]:
import pandas as pd 
import numpy as np
import math as math
input_df = pd.read_csv('../data/train/input_2023_w01.csv')
output_df = pd.read_csv('../data/train/output_2023_w01.csv')
supplemental_df = pd.read_csv('../data/supplementary_data.csv')
pd.set_option('display.max_columns', None)

df = pd.merge(input_df, supplemental_df, on=['game_id','play_id'], how='left')
print(df.head())

# Filtering out Zone coverages
df_man =  df[~df['team_coverage_type'].str.contains("ZONE")]
print(df_man['team_coverage_type'].value_counts())


      game_id  play_id  player_to_predict  nfl_id  frame_id play_direction  \
0  2023090700      101              False   54527         1          right   
1  2023090700      101              False   54527         2          right   
2  2023090700      101              False   54527         3          right   
3  2023090700      101              False   54527         4          right   
4  2023090700      101              False   54527         5          right   

   absolute_yardline_number player_name player_height  player_weight  \
0                        42  Bryan Cook           6-1            210   
1                        42  Bryan Cook           6-1            210   
2                        42  Bryan Cook           6-1            210   
3                        42  Bryan Cook           6-1            210   
4                        42  Bryan Cook           6-1            210   

  player_birth_date player_position player_side         player_role      x  \
0        1999-09-07 

/var/folders/0v/7v6b_kx92wn32v08jvcl87_h0000gn/T/ipykernel_28647/735440583.py:6: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplemental_df = pd.read_csv('../data/supplementary_data.csv')


In [3]:
df_man['x_player_ball_dist'] = df_man['ball_land_x']-df_man['x']
df_man['y_player_ball_dist'] = df_man['ball_land_y']-df_man['y']
# Calculate Euclidean distance from player to ball landing location
df_man['euclidean_dist_to_ball'] = np.sqrt(df_man['x_player_ball_dist']**2 + df_man['y_player_ball_dist']**2)
#print(df_man.head())
#print(df_man[df_man['play_id'] == 2389][['player_name', 'player_position', 'possession_team', 'player_to_predict']])

#Filter out non predicted players


df_man_pred = df_man[(df_man['player_to_predict'] == True) & df_man['player_position'].isin(['CB', 'S', 'DB', 'FS', 'SS', 'WR'])]

# Get the WR coordinates for each play
wr_coords = df_man_pred[df_man_pred['player_position'] == 'WR'][['game_id', 'play_id', 'frame_id', 'x', 'y']].rename(
    columns={'x': 'wr_x', 'y': 'wr_y'}
)

# Merge back to get WR coordinates for all players (including defenders)
df_man_pred = df_man_pred.merge(
    wr_coords,
    on=['game_id', 'play_id', 'frame_id'],
    how='left'
)

# Now calculate angle to receiver
df_man_pred['angle_to_receiver'] = np.degrees(np.arctan2(
    df_man_pred['wr_y'] - df_man_pred['y'],
    df_man_pred['wr_x'] - df_man_pred['x']
)) % 360
#pd.set_option('display.max_rows', None)
df_man_pred = df_man_pred[(df_man_pred['angle_to_receiver'].notna()) | (df_man_pred['player_position'] == 'WR')]

# Calculate the difference between where they're facing and where the receiver is
angle_diff = np.abs(df_man_pred['o'] - df_man_pred['angle_to_receiver'])

# Handle wraparound (e.g., 359° and 1° are only 2° apart, not 358°)
angle_diff = np.minimum(angle_diff, 360 - angle_diff)

# Consider "facing receiver" if within 22.5 degrees
df_man_pred['facing_receiver'] = angle_diff <= 22.5
print(df_man_pred[['player_name', 'player_position', 'angle_to_receiver', 'o', 'facing_receiver']].head(1000))



       player_name player_position  angle_to_receiver       o  facing_receiver
64     Justin Reid              SS         188.761593  282.47            False
65     Justin Reid              SS         188.930590  280.92            False
66     Justin Reid              SS         189.412629  280.92            False
67     Justin Reid              SS         189.908903  280.92            False
68     Justin Reid              SS         190.421850  281.96            False
...            ...             ...                ...     ...              ...
1282  Brian Branch              SS          58.209283   47.76             True
1283  Brian Branch              SS          60.888195   44.96             True
1284  Brian Branch              SS          63.748527   40.51            False
1285  Brian Branch              SS          66.586920   36.53            False
1286  Brian Branch              SS          68.932616   33.36            False

[1000 rows x 5 columns]


/var/folders/0v/7v6b_kx92wn32v08jvcl87_h0000gn/T/ipykernel_28647/1467738328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_man['x_player_ball_dist'] = df_man['ball_land_x']-df_man['x']
/var/folders/0v/7v6b_kx92wn32v08jvcl87_h0000gn/T/ipykernel_28647/1467738328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_man['y_player_ball_dist'] = df_man['ball_land_y']-df_man['y']
/var/folders/0v/7v6b_kx92wn32v08jvcl87_h0000gn/T/ipykernel_28647/1467738328.py:4: SettingWithCopyWarning: 
A value is trying t